For this Notebook the is needed only the library "Transformers" but we reccomend to install all the requirements present into the gitHub repository.

In [ ]:
! pip install transformers 

# Models Inference

This notebook provides an overview of the code generation capabilities of the models that have been pre-trained and fine-tuned on offensive PowerShell code generation. It outlines two methods of utilising these models: the first method employs the API provided by HuggingFace, which involves the models and dataset being uploaded; the second method downloads the models directly, requiring a minimum of 3 GB of free space on the user's disk.
Furthermore, the models may be tested using the graphical user interface (GUI) provided on the HuggingFace repository. This repository can be accessed via the following link: https://huggingface.co/collections/dessertlab/the-power-of-words-generating-powershell-attacks-from-66223c3e6cd34bb31ce38a69.


## Online Inference
In order to utilise the HuggingFace API for inference, it is necessary for the user to be logged into HuggingFace. Should the user be unwilling to log in, they may alternatively download the requisite models via the code provided below. 

In [ ]:
import requests

READ_TOKEN = "[HF TOKEN]"                                                                                       # insert your HF token of use the CLI HuggingFace LogIn
MODEL = "dessertlab/offensive-powershell-CodeGPT-small"
MODEL = "dessertlab/offensive-powershell-codegen-350M-multi"
MODEL = "dessertlab/offensive-powershell-codet5p-220m-py"                                                       # Choose one of the three models 
API_URL = f"https://api-inference.huggingface.co/models/{MODEL}"
HEADERS = {"Authorization": f"Bearer {READ_TOKEN}"}

def query(payload):
	response = requests.post(API_URL, headers=HEADERS, json=payload)
	return response.json()

if __name__ == "__main__":
    
    if("codet5p" in MODEL):
        output = query({
            "inputs": "System Information with WMI.",
            "options": {"wait_for_model": True}
        })
    else: #codegpt and codegen
        output = query({
            "inputs": "System Information with WMI. <s>",
            "parameters": {"return_full_text": False},
            "options": {"wait_for_model": True}
        })
    
    print(output)
    
    #skip undecodable characters
    output[0]['generated_text'] = output[0]['generated_text'].replace('�', ' ')

    with open('output.txt', 'w') as f:
        f.write(output[0]['generated_text'])


## In place inference

### CodeT5 plus


In [9]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM  
checkpoint_gpt = "dessertlab/offensive-powershell-CodeGPT-small"
MODEL = "dessertlab/offensive-powershell-codegen-350M-multi"
checkpoint_t5p = "dessertlab/offensive-powershell-codet5p-220m-py"  

tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=checkpoint_t5p)
model_t5p = AutoModelForSeq2SeqLM.from_pretrained(pretrained_model_name_or_path=checkpoint_t5p)


prompt = input("Please insert your prompt")
prompt = 'Execute PowerShell code to open the Windows calculator using remote command execution' #SAMPLE PROMPT
tokenized_outputs = tokenizer(
    prompt,
    max_length = 512, 
    truncation = True,  
    padding = 'max_length', 
    return_tensors = 'pt', 
)

output = model_t5p.generate(tokenized_outputs['input_ids'],
                            max_length=20, 
                            num_beams=5,    
                            early_stopping=True)

decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)

print("Generated code:", decoded_output)


Generated code: $code = {Invoke-Expression -Command "calc.exe"}; Invoke-Command


### CodeGPT

In [32]:
from transformers import AutoTokenizer, AutoModelForCausalLM
checkpoint_gpt = "dessertlab/offensive-powershell-CodeGPT-small"
checkpoint_gen = "dessertlab/offensive-powershell-codegen-350M-multi"
checkpoint_t5p = "dessertlab/offensive-powershell-codet5p-220m-py"  

tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=checkpoint_gpt)
model_gpt = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path=checkpoint_gpt)
tokenizer.padding_side = 'left'

prompt = input("Please insert your prompt")
prompt = 'Execute PowerShell code to open the Windows calculator using remote command execution. ' #SAMPLE PROMPT

tokenized_outputs = tokenizer(
    prompt,
    max_length = 200, 
    truncation = True,  
    padding = 'max_length',
    return_tensors = 'pt', 
)

output = model_gpt.generate(tokenized_outputs['input_ids'],   
                            max_length=512,
                            num_return_sequences=1
                        )

decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)

# Output cleaning

prompt_position = decoded_output.find(prompt)

# If the prompt is found, slice the generated text to remove it
if prompt_position != -1:
    generated_part = decoded_output[prompt_position + len(prompt):]
else:
    generated_part = decoded_output  

print("Generated Code without Prompt:")
print(generated_part.strip())  

Generated code: Execute PowerShell code to open the Windows calculator using remote command execution.  $code = {Invoke-Expression -Command "calc.exe"}; Invoke-Command -ScriptBlock $code
Generated Code without Prompt:
$code = {Invoke-Expression -Command "calc.exe"}; Invoke-Command -ScriptBlock $code
